Flash Technical Case Analysis: Mastercard Reporting (QMR) - Active Card Follow-Up Quarterly
--
In this notebook I intend use the queries built with SQL for details analysis, generating insights and recomentations.

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick # Important for formatting the percentage axis

def chart_barv(x_data, y_bar_data, title_name, y_bar_label = "", x_label = "", bar_color='#986A71'):
    """
    Plots a bar chart

    Args:
        x_data (pd.Series): Series for the x-axis (datetime objects).
        y_bar_data (pd.Series): Series for the bar chart data (absolute values).
        title_name (str): The chart's title.
        y_bar_label (str): The label for the primary y-axis (bars).
        x_label (str): The label for the x-axis.
        bar_color (str): Color for the bars.
    """
    fig, ax = plt.subplots(figsize=(12, 6))

    # Bar Chart
    vbars = ax.bar(x_data, y_bar_data, color=bar_color)
    ax.set_ylabel(y_bar_label, fontsize=12)
    #ax.tick_params(axis='y')
    ax.set_title(title_name, fontsize=14, pad=20)

    # data rotule
    ax.bar_label(vbars, fmt='{:,.0f}', padding=3)
                 
    # border
    ax.spines['top'].set_color('lightgray')       
    ax.spines['right'].set_color('lightgray')     
    ax.spines['bottom'].set_color('lightgray')    
    ax.spines['left'].set_color('lightgray') 

    # Adjust layout to make sure nothing is clipped
    fig.tight_layout()
    plt.show()

In [0]:
def import_query(path):
        with open(path, "r") as openFile:
            query = openFile.read()
            df = spark.sql(query)
            return df

In [0]:
df_quarter_result = import_query("quarter_results.sql")
display(df_quarter_result)

It is possible to see that there is a trend in growth KPIs, below we will analyze it in more detail.

### 1. Overall Card Growth

In [0]:
# 1. Generating df in pandas
dfp_quarter_result = df_quarter_result.toPandas()
dfp_quarter_result = dfp_quarter_result.sort_values(by='quarter_start_date')

# Extract columns into variables for clarity
x_values = dfp_quarter_result['quarter']
y_bar_values = dfp_quarter_result['qty_cards_end_quarter_total']
# 2. Ploting chart
chart_barv(
    x_data=x_values,
    y_bar_data=y_bar_values,
    title_name="Total Cards and Quarter-over-Quarter"
)

(+) High Growth KPIs

(+) Variation of Cards Temporarily Blocked Increased (Q2 to Q3 is smaller than Q1 to Q2)

(-) Cards Terminated During Quarter with Growing High

#### 1.1 Temporarily Blocked Analysis

In [0]:
df_quarter_result_tb = import_query("temporarily_blocked.sql")
display(df_quarter_result_tb )

In [0]:
# 1. Generating df in pandas
dfp_quarter_result_tb = df_quarter_result_tb.toPandas()
dfp_quarter_result_tb = dfp_quarter_result_tb .sort_values(by='quarter_start_date')

# Extract columns into variables for clarity
x_values = dfp_quarter_result_tb['quarter']
y_bar_values = dfp_quarter_result_tb['temp_blocked_not_reins_purged']
# 2. Ploting chart
chart_barv(
    x_data=x_values,
    y_bar_data=y_bar_values,
    title_name="Total Cards Temporarily Blocked",
    bar_color='#F3ABB3'
)

### 2. Card Model Analysis

#### 2.1 Implementation Contactless Cards

In [0]:
df_model = import_query("analysis_model.sql")
display(df_model)

In [0]:
import numpy as np

pdf_model = df_model.toPandas()
df_pivot = pdf_model.pivot(index="quarter", columns="card_model", values="qty_cards_end_quarter")

color_map = {
    'CONTACTLESS': '#F3ABB3',
    'PIN': '#986A71'
}

# Creating stacked chart
fig, ax = plt.subplots(figsize=(10, 6))

# Initialize the 'bottom' for each bar
bottom = np.zeros(len(df_pivot))

# Iterate over each card model (the columns of our df_pivot)
for model in df_pivot.columns:
    # Plot the bar for the current model, on top of the previous bars ('bottom')
    ax.bar(df_pivot.index, df_pivot[model], bottom=bottom, label=model, color=color_map.get(model))
    # Update the 'bottom' for the next iteration
    bottom += df_pivot[model]

# Customization and Chart Labels
ax.set_title("Card Base Evolution by Model")
ax.legend(title="Card Model", bbox_to_anchor=(1.05, 1), loc='upper left')

# Add labels with the total value and % contactless over PIN on top of each bar
for i, total in enumerate(bottom):
    contactless = df_pivot['CONTACTLESS'].iloc[i] if 'CONTACTLESS' in df_pivot.columns else 0
    pin = df_pivot['PIN'].iloc[i] if 'PIN' in df_pivot.columns else 0
    percentage = (contactless / (contactless + pin) * 100) if pin != 0 else 0
    ax.text(i, total + 5000, f'{int(total):,}\n{percentage:.2f}%', ha='center') # +5000 to give some space

# Border
ax.spines['top'].set_color('lightgray')       
ax.spines['right'].set_color('lightgray')     
ax.spines['bottom'].set_color('lightgray')    
ax.spines['left'].set_color('lightgray') 

plt.tight_layout() # Adjust the layout to prevent the legend from being cut off
plt.show()

#### 2.2 Card Analysis of Type: PIN Validity in Each Quarter

In [0]:
df_card = import_query("analysis_card_validity.sql")
display(df_card)

In [0]:
# 1. Generating pandas DataFrame
dfp_card = df_card.toPandas()
dfp_card = dfp_card.sort_values(by='quarter_start_date')

# Check the column names
print(dfp_card.columns)

# Extract columns into variables for clarity
x_values = dfp_card['quarter']
y_bar_values = dfp_card['qty_cards_end_quarter']

# 2. Plotting chart
chart_barv(
    x_data=x_values,
    y_bar_data=y_bar_values,
    title_name="Quarterly evolution of the total number of cards expiring in each quarter of the PIN type"
)